In [66]:
# todo
# 1. read train_edges from disk
# 2. create graph
# 3. setup params
# 4. generate temporal walks
# 5. train word2vec

In [132]:
import random
import pickle
import pandas as pd
import networkx as nx
from gensim.models import Word2Vec

seed = 42
random.seed(seed)

In [68]:
train_edges_file = '../datasets/collegemsg/train_edges.pkl'

with open(train_edges_file, 'rb') as te:
    train_edges = pickle.load(te)

In [69]:
graph = nx.MultiDiGraph()
graph.add_edges_from(train_edges)

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 2,
 1,
 0,
 3,
 3,
 4,
 0,
 1,
 2,
 0,
 5,
 3,
 4,
 6,
 7,
 1,
 8,
 9,
 10,
 5,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 11,
 12,
 0,
 1,
 0,
 2,
 0,
 1,
 6,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 7,
 3,
 2,
 4,
 3,
 4,
 5,
 0,
 1,
 0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 2,
 11,
 0,
 12,
 6,
 7,
 8,
 13,
 14,
 15,
 0,
 16,
 17,
 18,
 19,
 20,
 9,
 0,
 0,
 10,
 1,
 2,
 3,
 11,
 12,
 13,
 14,
 15,
 16,
 2,
 21,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 17,
 18,
 19,
 20,
 22,
 21,
 1,
 8,
 22,
 23,
 23,
 24,
 24,
 0,
 25,
 1,
 26,
 27,
 25,
 28,
 26,
 9,
 11,
 29,
 27,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 3,
 4,
 5,
 2,
 6,
 3,
 0,
 0,
 0,
 0,
 1,
 4,
 0,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 1,
 4,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 5,
 6,
 0,
 1,
 0,
 2,
 3,
 0,
 0,
 4,
 7,
 0,
 

In [109]:
N = len(graph.nodes)
omega = 10 # context window size
L = 80 # max length of walk
beta = 25 * N * (L - omega + 1)

In [110]:
C = 0
temporal_walks = []
edges = list(graph.edges(data=True))

In [ ]:
while beta - C > 0:
    walk = []
    first_edge = random.sample(edges, k=1) # unbiased first edge 
    u, v = first_edge[0][:2]
    walk.append(u)
    walk.append(v)
    vtime = first_edge[0][2]['timestamp']
    
    for i in range(L - 2):
        vneighbors = list(graph.out_edges(v, data=True))
        temp_neighbors = list(filter(lambda x: x[2]['timestamp'] >= vtime, vneighbors))
        if temp_neighbors:
            next_edge = random.sample(temp_neighbors, k=1)[0]
            v, vtime = next_edge[1], next_edge[2]['timestamp']
            walk.append(v)
            
    if len(walk) >= omega:
        temporal_walks.append(walk)
        C += len(walk) - omega + 1

In [139]:
temporal_walks_file = '../datasets/collegemsg/temporal_walks.pkl'

with open(temporal_walks_file, 'wb') as file:
    pickle.dump(temporal_walks, file)